# Prueba

100 canciones de los billboard - Prueba de scraping 

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
# Create a Chrome WebDriver instance
driver = webdriver.Chrome(options=options)

# URL de la página de Billboard
url = 'https://www.billboard.com/charts/hot-100'

# Abrir la página en el navegador controlado por Selenium
driver.get(url)

# Esperar a que la página se cargue completamente
time.sleep(5)

# Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
soup = BeautifulSoup(driver.page_source, 'html.parser')

titulos = soup.find_all('h3', id='title-of-a-story', class_='a-no-trucate')
artistas = soup.find_all('span', class_='a-no-trucate')

contador=0

# Imprimir los nombres de las canciones y artistas
if titulos and artistas:
    for titulo, artista in zip(titulos, artistas):
      contador = contador+1
      nombre_cancion= titulo.text.strip()
      nombre_artista = artista.text.strip()
      print(f'Canción: {nombre_cancion} - Artista: {nombre_artista}')

# Cerrar el navegador controlado por Selenium
driver.quit()

Canción: Fortnight - Artista: Taylor Swift Featuring Post Malone
Canción: Million Dollar Baby - Artista: Tommy Richman
Canción: A Bar Song (Tipsy) - Artista: Shaboozey
Canción: Espresso - Artista: Sabrina Carpenter
Canción: Beautiful Things - Artista: Benson Boone
Canción: Lose Control - Artista: Teddy Swims
Canción: Too Sweet - Artista: Hozier
Canción: Like That - Artista: Future, Metro Boomin & Kendrick Lamar
Canción: I Can Do It With A Broken Heart - Artista: Taylor Swift
Canción: Down Bad - Artista: Taylor Swift
Canción: Euphoria - Artista: Kendrick Lamar
Canción: Saturn - Artista: SZA
Canción: Who's Afraid Of Little Old Me? - Artista: Taylor Swift
Canción: So Long, London - Artista: Taylor Swift
Canción: Lovin On Me - Artista: Jack Harlow
Canción: My Boy Only Breaks His Favorite Toys - Artista: Taylor Swift
Canción: Push Ups - Artista: Drake
Canción: But Daddy I Love Him - Artista: Taylor Swift
Canción: We Can't Be Friends (Wait For Your Love) - Artista: Ariana Grande
Canción: Flo

# Genero - ROCK Y METAL

GENERO - ROCK Y METAL

In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from google.cloud import firestore
import os


# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# URL de la página a realizar scraping
urls = [
    'https://www.officialcharts.com/charts/rock-and-metal-singles-chart/20200605/111/',
    
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):

    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes, hay algunas imágenes que deben descartarse
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] != '/fallback/artwork-pink.svg']

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('rock').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y la elimina
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('rock').add(datos)

    return datos_completos


# Almacenar los datos en Firestore y eliminar las imágenes correspondientes
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios = procesar_pagina(url)

    for data in datos_audios:
        print(data)

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/rock-and-metal-singles-chart/20200605/111/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview112/v4/c9/ef/3b/c9ef3b6e-39a8-4b8e-904a-5bd6df38d17e/mzaf_8795282742978672195.plus.aac.p.m4a', 'songTitle': 'BOHEMIAN RHAPSODY', 'songSinger': 'QUEEN', 'image': 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2023-06/Bohemian-Rhapsody%20artwork.jpg?itok=YMbgjdSo'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview126/v4/b0/6d/e4/b06de449-cdee-6403-2168-4ee5ed6c0bed/mzaf_4942182476222230739.plus.aac.p.m4a', 'songTitle': "DON'T STOP ME NOW", 'songSinger': 'QUEEN', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/1b/71/65/1b7165f3-4830-04b2-7bc6-b59d1f5fa2e0/cover_4099885532375.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview112/v4/66/e9/46/66e9462c-bfc8-a92f-6917-da0da7c11b60/mz

# GENERO - POP

GENERO POP

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from google.cloud import firestore
import os

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/end-of-year-singles-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('/fallback/artwork-cobalt.svg', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2023-06/good%20love%20-%20hannah%20laing%20and%20roro.jpg?itok=jIq9a6hj')]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('pop').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('pop').add(datos)
        
    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_pop = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_pop, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/end-of-year-singles-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/3e/57/ef/3e57efa9-0572-82f5-0e92-ea5fef33982a/mzaf_270950342224133606.plus.aac.p.m4a', 'songTitle': 'FLOWERS', 'songSinger': 'MILEY CYRUS', 'image': 'https://is4-ssl.mzstatic.com/image/thumb/Music123/v4/9e/80/c7/9e80c757-6994-4338-9e79-b92d5f75f788/196589561725.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview126/v4/2d/04/3f/2d043fa1-3e9b-7842-0784-62cbb068177e/mzaf_13156766899654212166.plus.aac.p.m4a', 'songTitle': 'SPRINTER', 'songSinger': 'DAVE & CENTRAL CEE', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music116/v4/af/51/4c/af514ce7-327b-05fe-fd1a-88b149d9859f/23UMGIM58381.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview112/v4/92/fa/c9/92fac9f2-98b2-afe0-44db-f4bd3ed930d2/mzaf_8712547703118428730.plus.aac.p.m4a', 's

# GENERO - HIP HOP

GENERO - HIP HOP

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os 
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/official-hip-hop-and-r-and-b-singles-chart/'
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] != '/fallback/artwork-pink.svg']

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('hiphop').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('hiphop').add(datos)

    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_hiphop = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_hiphop, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()

Procesando página 1: https://www.officialcharts.com/charts/official-hip-hop-and-r-and-b-singles-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview211/v4/bd/d8/63/bdd8638e-5a00-beb6-b50c-afb2f757928a/mzaf_9774430316153724355.plus.aac.p.m4a', 'songTitle': 'PUSH UPS', 'songSinger': 'DRAKE', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music211/v4/54/d3/5d/54d35d13-dfab-74ab-b3e7-e37fc75337ad/23UM1IM56240.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/2c/59/69/2c59694d-674b-f702-8d7d-010c32d86d72/mzaf_13580580607794748358.plus.aac.p.m4a', 'songTitle': 'WHATEVER SHE WANTS', 'songSinger': 'BRYSON TILLER', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music116/v4/6d/c3/0c/6dc30cf7-86ee-5e87-8703-d0eb4fbddbdd/196871828352.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/b3/e2/be/b3e2beed-3c15-2822-a8d8-9c29f80e1375/mzaf_17758556019707644982.

# GENERO - JAZZ

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/jazz-and-blues-albums-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('/fallback/artwork-cobalt.svg', '/fallback/artwork-pink.svg', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-04/CS1011501-01A-BIG.jpg?itok=SgkkgaEE')]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('jazz').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('jazz').add(datos)

    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_jazz = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_jazz, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/jazz-and-blues-albums-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview126/v4/a4/2e/2c/a42e2c0f-3e34-07ed-96eb-b99c96c4c656/mzaf_15422960049303925130.plus.aac.p.m4a', 'songTitle': 'BEWITCHED', 'songSinger': 'LAUFEY', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/1f/de/6c/1fde6cb4-9703-c214-cdc6-8ecf7d0f0aa0/197189040030.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview125/v4/84/70/a6/8470a603-7626-727c-e93b-ea3ff84e4ba9/mzaf_13444400126636321976.plus.aac.p.m4a', 'songTitle': 'FRANK', 'songSinger': 'AMY WINEHOUSE', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music124/v4/6e/9a/40/6e9a4018-62c9-ff50-da03-11216f1e56f4/00602547456397.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview113/v4/07/c4/5d/07c45dd9-1c4c-db5d-e0d8-fb658e0417ed/mzaf_2148636893492510145.plus.aac.p.m4a', 'songTi

# GENERO - COUNTRY

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/country-artists-albums-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-02/the%20dead%20south.jpg?itok=D4edxUl8','https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-04/Taylor%20Swift%20-%20Taylor%20Swift.jpg?itok=CwQH-r-I','/fallback/artwork-pink.svg', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-03/BEYONCE-COWBOY-CARTER-ACT-2-ALBUM-COVER.jpg?itok=5nLVbojr') ]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('country').stream()]
    
    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('country').add(datos)
            
    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_country = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_country, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/country-artists-albums-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview122/v4/c7/0f/a4/c70fa463-b641-df92-4e0a-ebbe1b44a4ef/mzaf_4490235806565373786.plus.aac.p.m4a', 'songTitle': "FEARLESS (TAYLOR'S VERSION)", 'songSinger': 'TAYLOR SWIFT', 'image': 'https://is2-ssl.mzstatic.com/image/thumb/Music125/v4/c3/d0/1c/c3d01c88-73e7-187e-fd62-e1744de979a6/21UMGIM09915.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview112/v4/6a/7d/e6/6a7de6f8-b180-f78f-0ace-0b7c67b95d33/mzaf_12753068218355006698.plus.aac.p.m4a', 'songTitle': 'DEEPER WELL', 'songSinger': 'KACEY MUSGRAVES', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music116/v4/18/27/9e/18279e4a-8aaa-caf9-614b-5bded6134648/24UMGIM01566.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview122/v4/ce/4c/55/ce4c55a3-d66a-0ae0-07a8-b5db1806d370/mzaf_1637139901

# GENERO - DANCE

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/dance-singles-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('https://coverartarchive.org/release/4ee0fab6-2f1b-4039-9b1a-2d36ef335103/38647616459-250.jpg', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2023-06/good%20love%20-%20hannah%20laing%20and%20roro.jpg?itok=jIq9a6hj')]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('dance').stream()]
    
    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('dance').add(datos)

    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_dance = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_dance, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/dance-singles-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/4f/c4/c8/4fc4c86f-a22c-273e-c8ca-dc2fc703a2f0/mzaf_14888286557390016671.plus.aac.p.m4a', 'songTitle': 'PRADA', 'songSinger': 'CASSO/RAYE/D-BLOCK EUROPE', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music126/v4/81/59/f7/8159f7b5-52c0-003b-92c2-04c98a6147a1/196871373258.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/44/0d/58/440d5818-53e0-6f9b-48fd-13ec125073c9/mzaf_3479700492205900383.plus.aac.p.m4a', 'songTitle': 'HAPPIER', 'songSinger': 'BLESSED MADONNA/DOUGLAS', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music116/v4/9d/b5/c7/9db5c79b-d70a-d617-a8fa-6599b6094eac/5054197941139.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/f1/8e/c8/f18ec848-fbb7-8281-d371-ac890f591e47/mzaf_2945936763702838207.plus.aac.p.

# STREAMING

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/albums-streaming-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-03/BEYONCE-COWBOY-CARTER-ACT-2-ALBUM-COVER.jpg?itok=5nLVbojr', 'https://coverartarchive.org/release/64f647d1-e315-457a-8ede-c77c2f859ac3/38547854839-250.jpg', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/%5Bdate%3Acustom%3AY%5D-%5Bdate%3Acustom%3Am%5D/Oasis%20What%27s%20The%20Story%20sleeve.jpg?itok=9QbxLkdT', 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-01/Forty%20Licks%20-%20The%20Rolling%20Stones.jpg?itok=w0a425MD')]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('streaming').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('streaming').add(datos)
    
    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_streaming = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_streaming, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()


Procesando página 1: https://www.officialcharts.com/charts/albums-streaming-chart/
{'audioSrc': 'https://video-ssl.itunes.apple.com/itunes-assets/Video211/v4/e3/e6/6c/e3e66c76-9456-51c3-84f3-fa8937109b65/mzvf_6279594060325485267.640x480.h264lc.U.p.m4v', 'songTitle': 'THE TORTURED POETS DEPARTMENT', 'songSinger': 'TAYLOR SWIFT', 'image': 'https://backstage.officialcharts.com/sites/default/files/styles/charted_item_artwork_medium/public/2024-02/taylor-swift-the-tortured-poets-department-album.jpg?itok=5-f4oL14'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview113/v4/6a/61/16/6a611671-ded8-7ba2-80d0-f904eef0c3f4/mzaf_7156450567638289108.plus.aac.p.m4a', 'songTitle': 'THE HIGHLIGHTS', 'songSinger': 'WEEKND', 'image': 'https://is2-ssl.mzstatic.com/image/thumb/Music114/v4/dc/e9/95/dce995ab-5287-cba4-4966-46d16bf1c072/21UMGIM06667.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview116/v4/6b/da/19/6bda19d9-5034-4d9e-8e7

# GENERO  - AFROBEAT

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/afrobeats-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('/fallback/artwork-cobalt.svg') ]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('afro').stream()]


    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('afro').add(datos)
    
    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_afro = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_afro, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()

Procesando página 1: https://www.officialcharts.com/charts/afrobeats-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview211/v4/94/99/4e/94994eae-e028-6305-1587-81f48855648f/mzaf_11727906038540491276.plus.aac.p.m4a', 'songTitle': 'JUMP', 'songSinger': 'TYLA/GUNNA/SKILLIBENG', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music211/v4/ae/da/75/aeda7587-4013-c657-46b4-a951133f55b3/196871967594.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview221/v4/16/83/80/168380a1-3c33-ecc6-4d29-a4face3cb0c3/mzaf_6660084936601597322.plus.aac.p.m4a', 'songTitle': 'LOVE ME JEJE', 'songSinger': 'TEMS', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music211/v4/e5/a2/55/e5a255b3-f83e-f67a-e20f-23962b18dc3a/196871994545.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview221/v4/94/3c/e5/943ce51f-1f43-175c-024c-18858c0ad8d2/mzaf_12765878852385799305.plus.aac.p.m4a', 'songTitle': 'ART

# MIXXED

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
from google.cloud import firestore

# Configurar Selenium para usar Chrome en Google Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Ejecutar en modo headless (sin ventana)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Ruta al ejecutable de ChromeDriver
chromedriver_path = "/usr/lib/chromium-browser/chromedriver"

# Crear una instancia de Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Lista de URLs de las páginas
urls = [
    'https://www.officialcharts.com/charts/singles-chart/'
    # Agrega aquí más URLs si es necesario
]

# Función para procesar una página y extraer los datos de los audios
def procesar_pagina(url):
    # Abrir la página en el navegador controlado por Selenium
    driver.get(url)

    # Esperar a que la página se cargue completamente
    time.sleep(5)

    # Crear el objeto BeautifulSoup con el HTML de la página cargada por Selenium
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Encontrar todos los elementos <audio>
    audios = soup.find_all('audio')

    # Buscar las imágenes
    imagenes = soup.find_all('img', class_='chart-image-large')

    # Lista para almacenar los datos de cada audio con su respectiva imagen
    datos_completos = []

    # Lista para almacenar las imágenes
    lista_imagenes = [imagen['src'] for imagen in imagenes if imagen['src'] not in ('/fallback/artwork-cobalt.svg, https://coverartarchive.org/release/4ee0fab6-2f1b-4039-9b1a-2d36ef335103/38647616459-250.jpg') ]

    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "beat-builder-firebase-adminsdk-epk59-4413e34457.json"
    db = firestore.Client()
    
    # Lista de títulos de canciones en Firestore
    titulos_en_firestore = [doc.to_dict()['songTitle'] for doc in db.collection('random').stream()]

    # Iterar sobre cada elemento <audio>
    for audio in audios:
        src = audio['src']
        titulo = audio['data-title']
        artista = audio['data-artist']

        # Verificar si la lista de imágenes está vacía
        if not lista_imagenes:
            # Si la lista está vacía, no hay más imágenes disponibles, entonces salir del bucle
            break

        # Tomar la primera imagen de la lista y eliminarla
        imagen_asignada = lista_imagenes.pop(0)

        datos= {'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada}

        # Agregar los datos al diccionario final
        datos_completos.append({'audioSrc': src, 'songTitle': titulo, 'songSinger': artista, 'image': imagen_asignada})

        print(datos)

        if titulo not in titulos_en_firestore: 
            db.collection('random').add(datos)
    
    return datos_completos

# Procesar cada URL
for idx, url in enumerate(urls, start=1):
    print(f'Procesando página {idx}: {url}')
    datos_audios_random = procesar_pagina(url)

    # Imprimir los datos de cada audio
    for audio_idx, data in enumerate(datos_audios_random, start=1):
        print(f'  Audio {audio_idx}:')
        print(f'  Src: {data["audioSrc"]}')
        print(f'  Título: {data["songTitle"]}')
        print(f'  Artista: {data["songSinger"]}')
        print(f'  Imagen: {data["image"]}')

# Cerrar el navegador controlado por Selenium
driver.quit()

Procesando página 1: https://www.officialcharts.com/charts/singles-chart/
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview211/v4/97/a2/b8/97a2b874-e3de-5658-f889-5d0eedb72d3f/mzaf_13627043593740003520.plus.aac.p.m4a', 'songTitle': 'ESPRESSO', 'songSinger': 'SABRINA CARPENTER', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music211/v4/cb/64/8c/cb648cf7-e7bb-bd00-efe0-d744312c29a8/24UMGIM32304.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview221/v4/3f/cc/c6/3fccc64d-b2e2-35dc-4a42-75548d3d80d7/mzaf_16462301712629842716.plus.aac.p.m4a', 'songTitle': 'FORTNIGHT', 'songSinger': 'TAYLOR SWIFT FT POST MALONE', 'image': 'https://is1-ssl.mzstatic.com/image/thumb/Music221/v4/6b/7d/61/6b7d61e4-e6f1-83bc-d645-463aa06b33c4/24UMGIM29563.rgb.jpg/247x247bb.jpg'}
{'audioSrc': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview211/v4/58/77/2b/58772b8f-f76e-faaf-31dd-db5d8c57376e/mzaf_7312578909270296724.plus.aac